In [1]:
import numpy as np
import pandas as pd
import sys
import os
from codebase.file_utils import save_obj, load_obj
from codebase.post_process import samples_to_df
import altair as alt
alt.data_transformers.disable_max_rows()

%load_ext autoreload
%autoreload 2

In [15]:
ppp = True
log_dir = "./log/FND/PPP/20200418_013828_FND_CFA_s1m2/"
data = load_obj('data', log_dir)


if ppp: 
    ps = load_obj('ps', log_dir)
    ps.keys()
else: 
    print("\n\nChecking data integrity...\n\n")
    complete_data = load_obj("complete_data", log_dir)

    model_posterior_samples = dict()
    model_posterior_samples[0] = load_obj('ps_0', log_dir)
    model_posterior_samples[1] = load_obj('ps_1', log_dir)
    model_posterior_samples[2] = load_obj('ps_2', log_dir)



In [16]:
data

{'D': array([[0, 0, 0, 0, 0, 1],
        [0, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 1, 1],
        ...,
        [0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 1, 0, 0, 0]]), 'N': 565, 'J': 6, 'K': 2}

In [5]:
print(open(log_dir+'model.txt').read())

data {
  int<lower=1> N;
  int<lower=1> K;
  int<lower=1> J;
  int<lower=0, upper=1> DD[N, J];
}

transformed data{
  cov_matrix[J] I_J = diag_matrix(rep_vector(1, J));
}

parameters {
  vector[J] alpha;
  matrix[3,K] beta_free; // 3 free eleements per factor
  matrix[J-3,K] beta_zeros; // 3 zero elements per factor
  matrix[N,J] yy;
  cov_matrix[J] Omega_cov;
  cholesky_factor_corr[K] L_Phi;

}

transformed parameters{
  matrix[J,K] beta;
  cov_matrix[J] Marg_cov;
  corr_matrix[K] Phi_cov;
  for(j in 1:J) {
    for (k in 1:K) beta[j,k] = 0;
  }
  // set the free elements
  for (k in 1:K) beta[1+3*(k-1) : 3+3*(k-1), k] = beta_free[1:3,k];
  // set the zero elements
  beta[4:J, 1] = beta_zeros[1:(J-3), 1];
  beta[1:(J-3), K] = beta_zeros[1:(J-3), K];
  
  Phi_cov = multiply_lower_tri_self_transpose(L_Phi);
  Marg_cov = beta * Phi_cov * beta'+ Omega_cov;
}

model {
  to_vector(beta_free) ~ normal(0, 1);
  to_vector(beta_zeros) ~ normal(0, 0.1);
  to_vector(alpha) ~ normal(0, 10);
  L_Phi

In [6]:
if ppp:
    num_chains = ps['alpha'].shape[1]
    num_samples = ps['alpha'].shape[0]
    for chain_number in range(ps['alpha'].shape[1]):
        for i in range(num_samples):
            sign1 = np.sign(ps['beta'][i,chain_number,0,0])
            sign2 = np.sign(ps['beta'][i,chain_number,3,1])
            ps['beta'][i,chain_number,:3,0] = ps['beta'][i,chain_number,:3,0] * sign1
            ps['beta'][i,chain_number,3:,1] = ps['beta'][i,chain_number,3:,1] * sign2

            if 'Phi_cov' in ps.keys():
                ps['Phi_cov'][i,chain_number,0,1] = sign1 * sign2 * ps['Phi_cov'][i,chain_number,0,1]
                ps['Phi_cov'][i,chain_number,1,0] = ps['Phi_cov'][i,chain_number,0,1]
    
else:
    num_chains = model_posterior_samples[0]['alpha'].shape[1]
    num_samples = model_posterior_samples[0]['alpha'].shape[0]
    for j in range(3):
        ps = model_posterior_samples[j]    
        for chain_number in range(ps['alpha'].shape[1]):
            for i in range(num_samples):
                sign1 = np.sign(ps['beta'][i,chain_number,0,0])
                sign2 = np.sign(ps['beta'][i,chain_number,3,1])
                ps['beta'][i,chain_number,:3,0] = ps['beta'][i,chain_number,:3,0] * sign1
                ps['beta'][i,chain_number,3:,1] = ps['beta'][i,chain_number,3:,1] * sign2

                if 'Phi_cov' in ps.keys():
                    ps['Phi_cov'][i,chain_number,0,1] = sign1 * sign2 * ps['Phi_cov'][i,chain_number,0,1]
                    ps['Phi_cov'][i,chain_number,1,0] = ps['Phi_cov'][i,chain_number,0,1]
        model_posterior_samples[j] = ps



In [21]:
samples_to_df(ps, 'alpha').head()

,index,val,param_name,cn,row
0,0,-1.196962,alpha,0,0
1,1,-1.109889,alpha,0,0
2,2,-1.109159,alpha,0,0
3,3,-1.115131,alpha,0,0
4,4,-0.899546,alpha,0,0


In [17]:
df = samples_to_df(ps, 'beta')

c1 = alt.Chart(df[df.cn==0]).mark_line(opacity=0.8).encode(
    alt.X('index:O'),
    alt.Y('val'),
#     alt.Color('cn:N'),
    row = 'row:O',
    column= 'col:O'
).properties(width = 200, height=100, title='Beta') 

c1.interactive()

alt.Chart(...)

In [14]:
num_chains = ps['alpha'].shape[1]
num_samples = ps['alpha'].shape[0]
for chain_number in range(ps['alpha'].shape[1]):
    for i in range(num_samples):
        sign1 = np.sign(ps['beta'][i,chain_number,0,0])
        sign2 = np.sign(ps['beta'][i,chain_number,3,1])
        if sign2 == -1:
            print(i, chain_number)#             ps['beta'][i,chain_number,:3,0] = ps['beta'][i,chain_number,:3,0] * sign1
#             ps['beta'][i,chain_number,3:,1] = ps['beta'][i,chain_number,3:,1] * sign2

#             if 'Phi_cov' in ps.keys():
#                 ps['Phi_cov'][i,chain_number,0,1] = sign1 * sign2 * ps['Phi_cov'][i,chain_number,0,1]
#                 ps['Phi_cov'][i,chain_number,1,0] = ps['Phi_cov'][i,chain_number,0,1]

19 0
42 0
62 0
63 0
64 0
65 0
66 0
67 0
68 0
69 0
70 0
77 0
78 0
79 0
80 0
81 0
82 0
83 0
84 0
85 0
86 0
87 0
88 0
89 0
90 0
91 0
92 0
131 0
132 0
135 0
138 0
166 0
168 0
170 0
180 0
181 0
184 0
185 0
187 0
188 0
189 0
190 0
191 0
192 0
200 0
201 0
202 0
203 0
204 0
205 0
206 0
207 0
210 0
211 0
212 0
213 0
214 0
0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 1
11 1
12 1
13 1
14 1
15 1
16 1
17 1
18 1
19 1
20 1
21 1
22 1
23 1
24 1
25 1
26 1
27 1
28 1
29 1
30 1
31 1
32 1
33 1
34 1
35 1
36 1
37 1
38 1
39 1
40 1
41 1
42 1
43 1
44 1
45 1
46 1
47 1
48 1
49 1
50 1
51 1
52 1
53 1
54 1
55 1
56 1
57 1
58 1
59 1
60 1
61 1
62 1
63 1
64 1
65 1
66 1
67 1
68 1
69 1
70 1
71 1
72 1
73 1
74 1
75 1
76 1
77 1
78 1
79 1
80 1
81 1
82 1
83 1
84 1
85 1
86 1
87 1
88 1
89 1
90 1
91 1
92 1
93 1
94 1
95 1
96 1
97 1
98 1
99 1
100 1
101 1
102 1
103 1
104 1
105 1
106 1
107 1
108 1
109 1
110 1
111 1
112 1
113 1
114 1
115 1
116 1
117 1
118 1
119 1
120 1
121 1
122 1
123 1
124 1
125 1
126 1
127 1
128 1
129 1
130 1
131 1
132

In [23]:
c1 = alt.Chart(samples_to_df(ps, 'alpha')).mark_line(opacity=0.8).encode(
    alt.X('index:O'),
    alt.Y('val'),
    alt.Color('cn:N'),
    row = 'row:O',
    column= 'cn:N'
).properties(width = 200, height=100) 

c1.interactive()

alt.Chart(...)